# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""18230401277205…","""../../../../da…",0.983033,"""15045121443104…","""13241794685013…","""94222233410073…",false
"""16033935980634…","""../../../../da…",0.144221,"""85284515543563…","""13241794685013…","""46731001084018…",true
"""74647538086818…","""../../../../da…",0.679745,"""45427490618953…","""52492652953686…","""94222233410073…",false
"""14538694201987…","""../../../../da…",1.0,"""85284515543563…","""26103756510234…","""94222233410073…",false
"""14991480644097…","""../../../../da…",0.718982,"""15045121443104…","""26103756510234…","""55632385340511…",true
"""13796615250506…","""../../../../da…",0.0,"""45427490618953…","""41766019574853…",null,false
"""15394687642812…","""../../../../da…",0.02333,"""45427490618953…","""26103756510234…","""55632385340511…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""17461025767877…","""../../../../da…",0.983033,"""84596884097457…","""FirstMortgage3…","""12669120635100…","""Rejected"""
"""95858584796897…","""../../../../da…",0.144221,"""92697825883121…","""FirstMortgage3…","""15038615897645…","""Accepted"""
"""26603185664675…","""../../../../da…",0.679745,"""35036618451052…","""MoneyMarketSav…","""12669120635100…","""Rejected"""
"""57477757960930…","""../../../../da…",1.0,"""92697825883121…","""BasicChecking""","""12669120635100…","""Rejected"""
"""47211480484296…","""../../../../da…",0.718982,"""84596884097457…","""BasicChecking""","""48533781320001…","""Accepted"""
"""82328645391127…","""../../../../da…",0.0,"""35036618451052…","""UPlusFinPerson…",null,"""Rejected"""
"""70928557545467…","""../../../../da…",0.02333,"""35036618451052…","""BasicChecking""","""48533781320001…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""99237720360233…","""../../../../da…",0.983033,"""13235136263636…","""FirstMortgage3…","""36855300967555…","""Rejected"""
"""17374838874684…","""../../../../da…",0.144221,"""13921839255404…","""FirstMortgage3…","""15703949077309…","""Accepted"""
"""19019558796776…","""../../../../da…",0.679745,"""15685291931819…","""MoneyMarketSav…","""36855300967555…","""Rejected"""
"""13007456445199…","""../../../../da…",1.0,"""13921839255404…","""BasicChecking""","""36855300967555…","""Rejected"""
"""31991447106417…","""../../../../da…",0.718982,"""13235136263636…","""BasicChecking""","""37830296464809…","""Accepted"""
"""38658239521581…","""../../../../da…",0.0,"""15685291931819…","""UPlusFinPerson…",null,"""Rejected"""
"""21153313054748…","""../../../../da…",0.02333,"""15685291931819…","""BasicChecking""","""37830296464809…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""17365703334939…",0.983033,"""16749446200245…","""FirstMortgage3…","""97007343656452…","""Rejected"""
"""15186627092586…",0.144221,"""70803600115574…","""FirstMortgage3…","""15866242989268…","""Accepted"""
"""10581512266795…",0.679745,"""16977705343030…","""MoneyMarketSav…","""97007343656452…","""Rejected"""
"""69388596778533…",1.0,"""70803600115574…","""BasicChecking""","""97007343656452…","""Rejected"""
"""15602511724838…",0.718982,"""16749446200245…","""BasicChecking""","""68338176198180…","""Accepted"""
"""73353172274930…",0.0,"""16977705343030…","""UPlusFinPerson…",null,"""Rejected"""
"""28674699094310…",0.02333,"""16977705343030…","""BasicChecking""","""68338176198180…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""c73023dda51de1…","""../../../../da…",0.983033,"""af486bc1815c72…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
"""5414b197fb87b3…","""../../../../da…",0.144221,"""3cdf7b1ae1342e…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
"""892f5d61a9215e…","""../../../../da…",0.679745,"""806ef34eb8237f…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
"""c3aa807d9e927a…","""../../../../da…",1.0,"""3cdf7b1ae1342e…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
"""c2d492f9a05073…","""../../../../da…",0.718982,"""af486bc1815c72…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
"""b21059221605c4…","""../../../../da…",0.0,"""806ef34eb8237f…","""UPlusFinPerson…",null,"""Rejected"""
"""80cbe489dcecd4…","""../../../../da…",0.02333,"""806ef34eb8237f…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
